## Purpose of this script:
Prepare dataset in a format to be used in animation to easily study and evaluate plays 

## How to use this script:
This code creates a CSV file that can be used with the library developed here - https://github.com/samirak93/Game-Animation. Once the CSV is created, it shoudl be loaded into the 'sample_data' folder to generate the animation

In [2]:
import boto3
import json
import sys
import csv
import pandas as pd

In [3]:
player_play_rect = pd.read_pickle('../common/data/player_play_rect.pkl')
ball_rect = pd.read_pickle('../common/data/ball_rect.pkl')

In [4]:
def animate_play_rect(gameID, playID, plays_df = player_play_rect, ball_df = ball_rect):
    player_df = plays_df[(plays_df['gameId'] == gameID) & (plays_df['playId'] == playID)].explode('coordinates').reset_index(drop = True)
    player_df['time'] = player_df.groupby(( player_df['nflId'] != player_df['nflId'].shift(1) ).cumsum()).cumcount()+1
    player_df['x'] = player_df.apply(lambda row: row.coordinates[0], axis = 1)
    player_df['y'] = player_df.apply(lambda row: row.coordinates[1], axis = 1)
    player_df = player_df[['x', 'y', 'teamType', 'jerseyNumber', 'time']]
    player_df.rename(columns = {'teamType': 'team_id', 'jerseyNumber': 'player_id'}, inplace = True)
    
    ball = ball_df[(ball_df['gameId'] == gameID) & (ball_df['playId'] == playID)].explode('ballTrack').reset_index(drop = True)
    ball['x'] = ball.apply(lambda row: row.ballTrack[0], axis = 1)
    ball['y'] = ball.apply(lambda row: row.ballTrack[1], axis = 1)
    ball['time'] = ball.index + 1
    ball['team_id'] = 'ball'
    ball['player_id'] = -99
    ball = ball[['x', 'y', 'team_id', 'player_id', 'time']]
    
    max_time = player_df.time.max()
    
    ball = ball[ball['time'] <= max_time]
    
    animation_df = pd.concat([player_df, ball], ignore_index = True)
    animation_df.sort_values(['time', 'team_id'], ascending = [True, True], inplace = True)
    
    animation_df.to_csv('sample_data/animation_{}_{}.csv'.format(gameID, playID), index = False)
        animation_df.to_csv('sample_data/animation_{}_{}.csv'.format(gameID, playID), index = False)    
    
    return animation_df

In [6]:
animate_play_rect(gameID = 2019090802, playID = 1100)

,x,y,team_id,player_id,time
0,86.27,32.68,away,71,1
104,85.94,26.46,away,72,1
416,86.60,24.31,away,87,1
624,85.95,41.44,away,14,1
728,86.11,31.30,away,76,1
...,...,...,...,...,...
1559,46.91,13.42,home,26,104
1663,76.82,22.42,home,94,104
1975,69.17,15.66,home,90,104
2079,81.84,29.52,home,36,104


In [6]:
animate_play_rect(gameID = 2019112405, playID = 51)

,x,y,team_id,player_id,time
71,84.33,23.89,away,93,1
213,84.17,19.61,away,97,1
284,84.16,33.31,away,55,1
355,81.14,28.47,away,59,1
426,84.19,26.25,away,95,1
...,...,...,...,...,...
1064,84.00,27.55,home,62,71
1277,80.59,22.66,home,86,71
1348,80.95,31.88,home,71,71
1490,74.10,22.26,home,10,71
